# Converting `NetCDF` datasets to `GeoTiff`

<a id='libraries'></a>
### Python libraries

In [2]:
import rioxarray
import xarray

<a id='read_data'></a>
### Read `NetCDF`

In [17]:
year = '2016'
xds = xarray.open_dataset(f'../data/dataset-satellite-land-cover/C3S-LC-L4-LCCS-Map-300m-P1Y-{year}-v2.1.1.nc')
# add CRS
xds.rio.set_crs("epsg:4326")
xds

<xarray.Dataset>
Dimensions:              (bounds: 2, lat: 64800, lon: 129600, time: 1)
Coordinates:
  * lat                  (lat) float64 90.0 90.0 89.99 ... -89.99 -90.0 -90.0
  * lon                  (lon) float64 -180.0 -180.0 -180.0 ... 180.0 180.0
  * time                 (time) datetime64[ns] 2016-01-01
Dimensions without coordinates: bounds
Data variables:
    lccs_class           (time, lat, lon) uint8 ...
    processed_flag       (time, lat, lon) float32 ...
    current_pixel_state  (time, lat, lon) float32 ...
    observation_count    (time, lat, lon) uint16 ...
    change_count         (time, lat, lon) uint8 ...
    crs                  int32 ...
    lat_bounds           (lat, bounds) float64 ...
    lon_bounds           (lon, bounds) float64 ...
    time_bounds          (time, bounds) datetime64[ns] ...
Attributes:
    id:                         C3S-LC-L4-LCCS-Map-300m-P1Y-2016-v2.1.1
    title:                      Land Cover Map of 2016
    summary:                    This dataset characterizes the land cover of ...
    type:                       C3S-LC-L4-LCCS-Map-300m-P1Y
    project:                    EC C3S Land cover
    references:                 https://cds.climate.copernicus.eu/
    institution:                UCLouvain
    contact:                    copernicus-support@ecmwf.int
    comment:                    
    Conventions:                CF-1.6
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Standard Nam...
    keywords:                   land cover classification,satellite,observation
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    EC C3S Land cover Data Policy
    naming_authority:           
    cdm_data_type:              grid
    TileSize:                   2025:2025
    tracking_id:                1c3c8172-906d-4ce0-9aa7-f3a9d9b4e021
    product_version:            2.1.1
    creation_date:              20181130T105436Z
    creator_name:               UCLouvain
    creator_url:                http://www.uclouvain.be/
    creator_email:              landcover-cci@uclouvain.be
    source:                     PROBA-V
    history:                    lc-sr-1.0, lc-classification-1.0,lc-user-tool...
    time_coverage_start:        20160101
    time_coverage_end:          20161231
    time_coverage_duration:     P1Y
    time_coverage_resolution:   P1Y
    geospatial_lat_min:         -90.0
    geospatial_lat_max:         90.0
    geospatial_lon_min:         null
    geospatial_lon_max:         null
    spatial_resolution:         300m
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.002778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.002778

<a id='save_data'></a>
### Save `GeoTIFF`

In [19]:
# Save GeoTIFF 
xds.rio.set_crs("epsg:4326")
data = xds["lccs_class"]
data.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
data.rio.to_raster(f'../data/dataset-satellite-land-cover/ESA_{year}_ipcc.tif')

<a id='export_data'></a>
### Export `GeoTIFF` to Earth Engine

In [5]:
year = '2016'
!earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id projects/soils-revealed/esa_landcover_ipcc/ESA_{year}_ipcc ../data/dataset-satellite-land-cover/ESA_{year}_ipcc.tif

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

W0504 15:22:10.695231 140736101159808 __init__.py:46] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/

In [10]:
import ee
ee.Initialize()

In [15]:
image = ee.Image('users/resourcewatchlandcover/esa_landcover_ipcc/ESA_2001_ipcc')

task = ee.batch.Export.image.toAsset(
    image = image,
    description = 'imageToAsset',
    assetId = 'projects/soils-revealed/esa_landcover_ipcc/ESA_2001_ipcc',
    scale = 300,
    pyramidingPolicy = {'b1': 'mean'},
    maxPixels = 1e13)
task.start()